In [45]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import numpy as np
import pypyodbc
import csv
import os
import tempfile
import statsmodels.formula.api as smf

In [26]:
puma_ref = pd.read_excel('http://www.nyc.gov/html/dcp/download/census/nyc2010census_tabulation_equiv.xlsx',header=3)
# alternatively, use shapefile from https://data.cityofnewyork.us/Housing-Development/Public-Use-Microdata-Areas-PUMA-/cwiz-gcty
puma_list = puma_ref['PUMA'].unique()[~np.isnan(puma_ref['PUMA'].unique())]

In [86]:
pums_data = pd.read_csv('ss14hny.csv')
pums_data['PUMA'] = pums_data[['PUMA00','PUMA10']].max(axis=1)

In [95]:
nyc_pums = pd.DataFrame()
for puma in puma_list:
    nyc_pums = nyc_pums.append(pums_data.query('PUMA == @puma'))
    print nyc_pums.shape[0],'rows after addition of PUMA',str(puma)

2532 rows after addition of PUMA 3704.0
4290 rows after addition of PUMA 3706.0
7271 rows after addition of PUMA 3705.0
9141 rows after addition of PUMA 3703.0
11081 rows after addition of PUMA 3708.0
13603 rows after addition of PUMA 3702.0
15801 rows after addition of PUMA 3707.0
18926 rows after addition of PUMA 3710.0
21257 rows after addition of PUMA 3701.0
24034 rows after addition of PUMA 3709.0
27707 rows after addition of PUMA 4017.0
30805 rows after addition of PUMA 4013.0
33408 rows after addition of PUMA 4003.0
36267 rows after addition of PUMA 4014.0
38410 rows after addition of PUMA 4018.0
41972 rows after addition of PUMA 4004.0
44269 rows after addition of PUMA 4007.0
46446 rows after addition of PUMA 4002.0
50660 rows after addition of PUMA 4009.0
53223 rows after addition of PUMA 4005.0
55807 rows after addition of PUMA 4006.0
57735 rows after addition of PUMA 4011.0
60478 rows after addition of PUMA 4008.0
63099 rows after addition of PUMA 4010.0
66109 rows after add

In [96]:
# nyc_pums[nyc_pums['PUMA']==4004].groupby(['YBL']).size()

In [98]:
print('Example median HH income in Williamsburg/Greenpoint')
nyc_pums[nyc_pums['PUMA10']==4001]['HINCP'].median()

Example median HH income in Williamsburg/Greenpoint


60000.0

In [99]:
print ('Median HH income in all NYC PUMAs')
nyc_pums['HINCP'].median()

Median HH income in all NYC PUMAs


52000.0

In [100]:
# nyc_pums.groupby(['YBL'])['HINCP'].median()

In [ ]:
# dependents to test: HINCP, RNTP, GRNTP, NPF, TAXP
# independents to test: YBL (ord), TEN (c), BDSP, BLD, VACS - perhaps as a filter?
# also of interest: MV

In [93]:
mod = smf.ols(formula='RNTP ~ YBL + C(BLD) + BDSP + C(PUMA) + C(TEN)', data = nyc_pums).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                   RNTP   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     481.5
Date:                Wed, 24 Feb 2016   Prob (F-statistic):               0.00
Time:                        11:24:41   Log-Likelihood:            -6.1594e+05
No. Observations:               79029   AIC:                         1.232e+06
Df Residuals:                   78963   BIC:                         1.233e+06
Df Model:                          65                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept       -6.307e+13   1.06e+14     